<a href="https://colab.research.google.com/github/songqsh/foo1/blob/master/src/transport_tableau.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transport problem
From Section 8.1 - tableau



In [1]:
# import package
import numpy as np
import numpy.linalg as la
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
import pandas as pd
import warnings
#ignore by message
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")

## Pivotizing

Pivotizing with $(i,j)$-entry means converting the matrix by multiple EROS so that the resulting matrix has all zeros in $j$-column, but one in $(i,j)$-entry. This is the key step in updating basic feasible solution.

In [2]:
#pivotize with (i,j)
def pivot(A, i, j):
  A[i] = A[i]/A[i,j] #scale to get one in (i,j)
  n_rows, _ = A.shape
  for k in range(n_rows):
    if k==i:
      continue # skip i-row
    A[k] = A[k] - A[i]*A[k,j] # replacement to get zero

## Update

In [3]:
def minratio(ratio_list):
  pivot_row = 0;
  minratio = np.inf;
  for i in range(len(ratio_list)):
    if ratio_list[i]>0 and ratio_list[i]<minratio:
      pivot_row = i+1
      minratio = ratio_list[i]
  if minratio is np.inf:
    return 0 #no leaving variable
  else:
    return pivot_row

def newpivot(M): #M is the tableau
  m, n = M.shape
  m = m-1 #number of constraints in the augmented form
  n = n-2 #number of variables in the augmented form
  optimal_test = min(M[0, range(1,n+1)])
  
  if optimal_test<0:
    pivot_col = np.argmin(M[0,range(1,n+1)])+1
  else:
    print(f'pass the optimal test')
    return(0)
  ratio_list = np.divide(M[range(1,m+1),-1], M[range(1, m+1),pivot_col])

  minratiotest = minratio(ratio_list)
  if minratiotest is 0:
    print(f'no leaving variable, here is the ratio list {ratio_list}')
    return(0)
  else:
    pivot_row = minratiotest
  return(pivot_row, pivot_col)

# Transport problem - BigM Simplex

In [4]:
A = np.array([[1., 1., 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 
              [0., 0., 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0., 0., 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0], 
              [1., 0., 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0., 1., 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0., 0., 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
              [0., 0., 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]])
b = np.array([[75., 125, 100, 80, 65, 70, 85]]).T

M=1000
c = np.array([464., 513, 654, 867, 352, 416,
              690, 791, 995, 682, 388, 685, 
              M, M, M, M, M, M, M])
m, n = A.shape
print(f'coefficient matrix is \n {pd.DataFrame(A)}')
print(f'right hand side vector is \n {b}')
print(f'coefficient of objective function is \n {c}')
print(f'n is {n} and m is {m}')

coefficient matrix is 
     0    1    2    3    4    5    6   ...   12   13   14   15   16   17   18
0  1.0  1.0  1.0  1.0  0.0  0.0  0.0  ...  1.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  1.0  1.0  1.0  ...  0.0  1.0  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  1.0  0.0  0.0  0.0  0.0
3  1.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  0.0  1.0  0.0  0.0  0.0
4  0.0  1.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0  0.0  1.0  0.0  0.0
5  0.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0
6  0.0  0.0  0.0  1.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  1.0

[7 rows x 19 columns]
right hand side vector is 
 [[75.00]
 [125.00]
 [100.00]
 [80.00]
 [65.00]
 [70.00]
 [85.00]]
coefficient of objective function is 
 [464.00 513.00 654.00 867.00 352.00 416.00 690.00 791.00 995.00 682.00
 388.00 685.00 1000.00 1000.00 1000.00 1000.00 1000.00 1000.00 1000.00]
n is 19 and m is 7


In [5]:
#initial tableau
M1 = np.append(np.zeros([m,1]), A, axis=1)
M1 = np.append(M1, b, axis=1)
M2 = np.append([1], c)
M2 = np.append(M2, [0])
AM = np.append([M2], M1, axis=0)
print(f'tableau after big M is \n {pd.DataFrame(AM)}')

tableau after big M is 
     0      1      2      3      4   ...      16      17      18      19     20
0  1.0  464.0  513.0  654.0  867.0  ...  1000.0  1000.0  1000.0  1000.0    0.0
1  0.0    1.0    1.0    1.0    1.0  ...     0.0     0.0     0.0     0.0   75.0
2  0.0    0.0    0.0    0.0    0.0  ...     0.0     0.0     0.0     0.0  125.0
3  0.0    0.0    0.0    0.0    0.0  ...     0.0     0.0     0.0     0.0  100.0
4  0.0    1.0    0.0    0.0    0.0  ...     1.0     0.0     0.0     0.0   80.0
5  0.0    0.0    1.0    0.0    0.0  ...     0.0     1.0     0.0     0.0   65.0
6  0.0    0.0    0.0    1.0    0.0  ...     0.0     0.0     1.0     0.0   70.0
7  0.0    0.0    0.0    0.0    1.0  ...     0.0     0.0     0.0     1.0   85.0

[8 rows x 21 columns]


In [6]:
#initialize the tableau
for i in range(1,8):
  pivot(AM, i, i+12)
print(f'initial tableau is \n {pd.DataFrame(AM)}')

initial tableau is 
     0       1       2       3       4   ...   16   17   18   19        20
0  1.0 -1536.0 -1487.0 -1346.0 -1133.0  ...  0.0  0.0  0.0  0.0 -600000.0
1  0.0     1.0     1.0     1.0     1.0  ...  0.0  0.0  0.0  0.0      75.0
2  0.0     0.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0     125.0
3  0.0     0.0     0.0     0.0     0.0  ...  0.0  0.0  0.0  0.0     100.0
4  0.0     1.0     0.0     0.0     0.0  ...  1.0  0.0  0.0  0.0      80.0
5  0.0     0.0     1.0     0.0     0.0  ...  0.0  1.0  0.0  0.0      65.0
6  0.0     0.0     0.0     1.0     0.0  ...  0.0  0.0  1.0  0.0      70.0
7  0.0     0.0     0.0     0.0     1.0  ...  0.0  0.0  0.0  1.0      85.0

[8 rows x 21 columns]


In [7]:
npivot = newpivot(AM)
while npivot != 0:
  print(f'new pivot is {npivot}')
  if npivot is not 0:
    pivot(AM, npivot[0], npivot[1])
    print(pd.DataFrame(AM))
    print(f'=======================')
    npivot = newpivot(AM)

new pivot is (4, 5)
    0      1       2       3       4   ...      16   17   18   19        20
0  1.0  112.0 -1487.0 -1346.0 -1133.0  ...  1648.0  0.0  0.0  0.0 -468160.0
1  0.0    1.0     1.0     1.0     1.0  ...     0.0  0.0  0.0  0.0      75.0
2  0.0   -1.0     0.0     0.0     0.0  ...    -1.0  0.0  0.0  0.0      45.0
3  0.0    0.0     0.0     0.0     0.0  ...     0.0  0.0  0.0  0.0     100.0
4  0.0    1.0     0.0     0.0     0.0  ...     1.0  0.0  0.0  0.0      80.0
5  0.0    0.0     1.0     0.0     0.0  ...     0.0  1.0  0.0  0.0      65.0
6  0.0    0.0     0.0     1.0     0.0  ...     0.0  0.0  1.0  0.0      70.0
7  0.0    0.0     0.0     0.0     1.0  ...     0.0  0.0  0.0  1.0      85.0

[8 rows x 21 columns]
new pivot is (6, 11)
    0      1       2      3       4   ...      16   17      18   19        20
0  1.0  112.0 -1487.0  266.0 -1133.0  ...  1648.0  0.0  1612.0  0.0 -355320.0
1  0.0    1.0     1.0    1.0     1.0  ...     0.0  0.0     0.0  0.0      75.0
2  0.0   -1.0     